### 주유소 데이터 수집 및 분석
* 엑셀 데이터를 사용하지 말고\
상표, 주유소명, 주소, 유종별 평균 가격(보통휘발유, 경유),\
부가정보(세차장, 충전소, 경정비, 편의점, 영업시간)를 가져와보자. (서울시 구별)
* 부가정보별 데이터 분석도 해보자.

* 서울 특별시 구 : 25개
* selenium을 사용하여 서울시 구별 데이터를 습득
* 각 구에 대한 주유소 정보를 습득하여 html 내 정보를 통해 유가, 부가정보, 영업시간, 업데이트 일시를 등록
* 부가정보의 세차장, 충전소, 경정비 등의 정보는 파일명 "service1_01.gif" or "_off.gif" 


In [ ]:
import pandas as pd
import json
import folium
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from tqdm.notebook import tqdm

import time

In [ ]:
gasStation = webdriver.Chrome(service=Service("../driver/chromedriver"))
gasStation.maximize_window()
gasStation.get("https://www.opinet.co.kr/searRgSelect.do")
gasStation.implicitly_wait(5)

In [ ]:
sido = gasStation.find_element(By.ID,"SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME,"option")
sido_names = [sido_name.get_attribute("value") for sido_name in sido_list]
sido_names = sido_names[1:]
sido.send_keys(sido_names[0])

sigungu = gasStation.find_element(By.ID, "SIGUNGU_NM0")   #select seoul
sigungu_list = sigungu.find_elements(By.TAG_NAME,"option")
sigungu_names = [sigungu_name.get_attribute("value") for sigungu_name in sigungu_list]
sigungu_names = sigungu_names[1:]

station_info_list = []

# district
for sigungu_name in tqdm(sigungu_names) :
    
    sigungu = gasStation.find_element(By.ID, "SIGUNGU_NM0")
    sigungu.send_keys(sigungu_name)
    gasStation.implicitly_wait(1)

    # number of gas stations
    station_count = gasStation.find_element(By.ID,"totCnt")
    station_count = int(station_count.text) 
    
    # body1 : gasoline, body2 : premium gasoline, body3 : disel, body4 : paraffin
    gasoline = gasStation.find_element(By.ID,"body1")       
    station_list= gasoline.find_elements(By.TAG_NAME, "a") 

    for count in range(station_count): 
    
        station_list[count].click()                                          

        # information gathering
        station_name    = gasStation.execute_script(f'return document.getElementById("{"os_nm"}").innerHTML;')                     # name
        station_number  = gasStation.execute_script(f'return document.getElementById("{"phn_no"}").innerHTML;')                    # number
        station_address = gasStation.execute_script(f'return document.getElementById("{"rd_addr"}").innerHTML;').split(maxsplit=2) # address
        station_brand   = gasStation.execute_script(f'return document.getElementById("{"poll_div_nm"}").innerHTML;')               # brand
        station_sigungu = station_address[1]
        station_address = station_address[2]

        station_price = [                                                                               # price
            gasStation.execute_script(f'return document.getElementById("{"b034_p"}").innerHTML;'),      # premium gasoline
            gasStation.execute_script(f'return document.getElementById("{"b027_p"}").innerHTML;'),      # gasoline
            gasStation.execute_script(f'return document.getElementById("{"d047_p"}").innerHTML;'),      # disel
            gasStation.execute_script(f'return document.getElementById("{"c004_p"}").innerHTML;')       # paraffin
        ]

        # popup detail information page
        station_detail_info = gasStation.find_element(By.ID,"os_dtail_info")    

        # additional service car wash, charging, maintenance, convenience, 24/7
        service_list = ["cwsh_yn", "lpg_yn", "maint_yn", "cvs_yn", "sel24_yn"]
        station_service = ["X"] * len(service_list)

        for service in service_list :                                           
            if 'off' not in station_detail_info.find_element(By.ID,service).get_attribute("src"):   station_service[service_list.index(service)]="O"

        # data merge 
        station_info = [station_sigungu, station_brand, station_name, station_address, station_number] 
        station_info.extend(station_price)
        station_info.extend(station_service)
        station_info_list.append(station_info)


In [ ]:
gasStation.quit()


주유소의 세부 정보 및 가격 정보를 가져오는 과정에 팝업 정보에서 
할인이나 행사 정보 사항이 많을 경우 팝업 창이 화면 밖으로 이탈하였고\
이 경우 find_element는 효과적이로 정보를 가져오지 못했다.\
팝업 창 display 여부를 효과적으로 변경하기 위한 방법들을 모색했다.

1. HTML 정보를 변경하여 팝업창 display 여부를 변경하지 못했다.
2. 지도화면 이동을 통해 팝업 창을 확인하는 방법은 부정확하고 안정성이 떨어진다.
3. 줌인/아웃을 하면 팝업 창이 화면 내부로 들어오는 점을 이용했지만 지도화면 로딩 시간, 위치의 부정확함으로 인한 오차가 발생했다.
4. HTML 정보상에 표시되는 팝업 창의 전시 위치 픽셀을 가공해서 활용하는 것은 어려워보인다.

팝업창이 화면에 보이지 않아도 HTML 상에는 정보가 있는 점을 이용해서 HTML 상에서 정보를 가져왔다.

In [ ]:
seoul_station_data = pd.DataFrame(station_info_list)
seoul_station_data.columns = ["구", "상호", "이름", "주소", "번호", "고급유", "휘발유", "경유", "등유", "세차장", "충전소", "정비소", "편의점", "24시"]

# 기름 가격 정보를 표기하기 위해 빈 데이터를 0으로 채우고, float 형식으로 변환
fuel_types = ["고급유", "휘발유", "경유", "등유"]
seoul_station_data[fuel_types] = seoul_station_data[fuel_types].fillna(0).replace('', 0).apply(lambda x: x.str.replace(',', '').astype(float))

writer = pd.ExcelWriter("../data/seoul_station_data.xlsx", engine='xlsxwriter')
seoul_station_data.to_excel(writer, sheet_name='Sheet1')
writer.close()


In [ ]:
seoul_gu_data_map = pd.pivot_table(data = seoul_station_data, index="구", values="등유", aggfunc=np.mean)

geo_path = "../data/skorea_municipalities_geo_simple.json"
geo_str = json.load(open(geo_path, encoding="utf-8"))

map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles="OpenStreetMap")
folium.Choropleth(geo_data=geo_str,
                  data=seoul_gu_data_map,
                  columns=[seoul_gu_data_map.index, '등유'],
                  key_on='feature.id',
                  fill_color='PuRd').add_to(map)
                              
map